# Clean Slate: Estimating offenses eligible for expungement under varying conditions
> Prepared by [Laura Feeney](https://github.com/laurafeeney) for Code for Boston's [Clean Slate project](https://github.com/codeforboston/clean-slate).

## Purpose
This notebook takes somewhat processed data from the Northwestern DA and answers the following questions, posed June 28, 2020. 

This analysis will be run a number of times: under existing regulation, and under varying conditions allowing additional offenses to be expunged.


1. How many people (under age 21) are eligible for expungement today? This would be people with only **one charge** that is not part of the list of ineligible offenses (per section 100J). 


2. How many people (under age 21) would be eligible based on only having **one incident** (which could include multiple charges) that are not part of the list of ineligible offenses?
 - How many people (under age 21) would be eligible based on only having **one incident** if only sex-based offenses or murder were excluded from expungement?
 

3. How many people (under age 21) would be eligible based on who has **not been found guilty** (given current offenses that are eligible for expungement)?
 - How many people (under age 21) would be eligible based on who has **not been found guilty** for all offenses except for murder or sex-based offenses?

-----

### Step 0
Import data, programs, etc.

-----

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 200)
import numpy as np
import regex as re
import glob, os
import datetime 
from datetime import date 
from collections import defaultdict, Counter

In [2]:
# processed individual-level data from NW district with expungability.

nw = pd.read_csv('../../data/processed/merged_nw.csv', encoding='cp1252',
                    dtype={'Analysis notes':str, 'extra_criteria':str, 'Expungeable': str}) 

nw['Expungeable'].value_counts(dropna=False)


Yes              55052
No               20013
NotApplicable      439
Attempt            220
NaN                  1
Name: Expungeable, dtype: int64

## Step 1
Add a few additional variables needed, look at summary stats

There are many offenses that are violations of the Code of Massachusetts Regulations (CMR) rather than a criminal offense. These include things like some driving or boating infractions (e.g., not having headlights on), or not having a hunting/fishing license. It's not clear whether these should be included at all. We'll run the analysis first including them at all stages, and then excluding them at all stages. 

In [3]:
### number of unique people
nw['Person ID'].nunique()

19753

In [4]:
### dates ###

reference_date = datetime.date(2020, 9, 1) # using "today.date() wouldn't be stable"

nw['Offense Date'] = pd.to_datetime(nw['Offense Date']).dt.date
nw['years_since_offense'] = (reference_date - nw['Offense Date'])/pd.Timedelta(1, 'D')/365

nw[nw['Offense Date']<datetime.date(1950, 1, 1)]

print("The earliest offense date is", min(nw['Offense Date']))
print("The max offense date is", max(nw['Offense Date']), "\n")

print(nw['years_since_offense'].describe())

print("\n There are a tail of dates that are probably wrong, but age is missing for most anyways,"
      + "so we typically exclude offenses without an age")

The earliest offense date is 1750-12-03
The max offense date is 2018-12-30 

count    74915.000000
mean         4.460396
std          2.798864
min          1.673973
25%          3.060274
50%          4.328767
75%          5.638356
max        269.926027
Name: years_since_offense, dtype: float64

 There are a tail of dates that are probably wrong, but age is missing for most anyways,so we typically exclude offenses without an age


In [5]:
#Manipulating the dataframe to Drop all CMR offenses and Drop CMR-related columns

print(f'There are {nw.shape[0]} total offenses including CMR.')

nw = nw.loc[nw['CMRoffense'] == 'no']
nw = nw.drop(columns = ['CMRoffense'])

print(f'After we drop CMR, there are {nw.shape[0]} total offenses.')

There are 75725 total offenses including CMR.
After we drop CMR, there are 75285 total offenses.


In [6]:
# distribution of # of charges

nw['num_offenses']=nw.groupby('Person ID')['Person ID'].transform('count')
print(nw['num_offenses'].describe())

#nw.loc[nw['num_offenses']<30].hist(column='num_offenses', bins=15)

count    75285.000000
mean        10.075752
std         18.230430
min          1.000000
25%          3.000000
50%          5.000000
75%         11.000000
max        202.000000
Name: num_offenses, dtype: float64


In [7]:
a = nw[nw['num_offenses']==1]['Person ID'].nunique()
print("# People with only 1 offense", a)

# People with only 1 offense 5026


In [8]:
print(nw['Expungeable'].value_counts())

Yes        55052
No         20013
Attempt      220
Name: Expungeable, dtype: int64


## Question 1
- How many people are eligible for expungement today? 
    - ---> *Only one charge*, Offense committed before 21st birthday, charge is not part of the list of ineligible offenses (per section 100J), charge is in the correct timeframe from today's date

We don't have misdemeanor / felony info, so will show the # that are more than 3 or more than 7 years from today.

----

In [9]:
def date_range(x):
    greater3 = x.loc[(x['years_since_offense'] > 3)]['Person ID'].nunique()
    greater7 = x.loc[(x['years_since_offense'] > 7)]['Person ID'].nunique()

    print(greater3, "occured more than 3 years before", reference_date)
    print(greater7, "occured more than 7 years before", reference_date)
    

In [10]:
eligible_all_ages = nw.loc[
    (nw['num_offenses']==1) &
    (nw['Expungeable'] != 'No') &
    (~nw['Age at Offense'].isnull())
]['Person ID'].nunique()

print("There are", eligible_all_ages, "individuals of any age with a single offense that is theoretically eligible for expungment.")

There are 3557 individuals of any age with a single offense that is theoretically eligible for expungment.


In [11]:
x = nw.loc[
    (nw['num_offenses']==1) &
    (nw['Expungeable'] != 'No') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21)
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA",
      "if we include 'attempts' as expungeable.")

date_range(x)

x.Disposition.value_counts(dropna=False)

There are 549 people with charges eligible for expungement in the NW district of MA if we include 'attempts' as expungeable.
392 occured more than 3 years before 2020-09-01
11 occured more than 7 years before 2020-09-01


c276s87 finding                     154
Dismissed at Request of Comm         96
Responsible                          80
Nolle Prosequi                       50
Dismissed Prior to Arraignment       42
Dismissed                            32
Dismissed on Payment                 28
Continued w/o Finding                26
NaN                                  25
Guilty                                6
Charge Handled as a Civil Charge      3
Delinquent                            3
Dismissed by Court                    3
Responsible Filed                     1
Name: Disposition, dtype: int64

In [12]:
x = nw.loc[
    (nw['num_offenses']==1) &
    (nw['Expungeable'] == 'Yes') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA,",
      " if we assume all 'attempts' are not expungeable.")

date_range(x)

There are 547 people with charges eligible for expungement in the NW district of MA,  if we assume all 'attempts' are not expungeable.
390 occured more than 3 years before 2020-09-01
11 occured more than 7 years before 2020-09-01


## Question 2: incidents rather than offenses

Luke Schissler & Laura Feeney

2. How many people (under age 21) would be eligible based on only having **one incident** (which could include multiple charges) that are not part of the list of ineligible offenses?
 - 2a. How many people (under age 21) would be eligible based on only having **one incident** if only sex-based offenses or murder were excluded from expungement?


Making the assumption based on the 06/26/20 meeting notes that an 'incident' is defined as the sum of all charges for an individual on a single day. In theory, someone could probably be charged in two separate instances on a single day, but this is likely rare. 

For this analysis, we assume that a single non-expungeable offense under the bundle of the incident makes the entire incident non-expungeable. Same for the analysis of murder or sex crimes.

In [13]:
#Data prep

#Number of incidents (separate offense dates per same ID)
nw['Incidents'] = (nw.groupby('Person ID')['Offense Date'].transform('nunique'))

# If an incident includes one offense that is not expungeable, we mark the entire incident as not expungeable.
#Attempts *are not* considered expungeable in this one. 
nw['Exp'] = nw['Expungeable']=="Yes"
nw['Inc_Expungeable_Attempts_Not'] = nw.groupby(['Person ID', 'Offense Date'])['Exp'].transform('min')

# If an incident includes one offense that is not expungeable, we mark the entire incident as not expungeable.
#Attempts *are* not considered expungeable in this one. 
nw['ExpAtt'] = (nw['Expungeable']=="Yes") | (nw['Expungeable']=="Attempt")
nw['Inc_Expungeable_Attempts_Are'] = nw.groupby(['Person ID', 'Offense Date'])['ExpAtt'].transform('min')

# If an incident includes an offense that is a murder and/or sex crime, we code the whole incident as regarding
# murder and/or sex.
nw['sm'] = (nw['sex'] == 1) | (nw['murder'] ==1)
nw['Incident_Murder_Sex'] = nw.groupby(['Person ID', 'Offense Date'])['sm'].transform('max')

#Sanity check to make sure I coded things right
#nw[['Person ID', 'Offense Date', 'murder', 'sex', 'Incident_Murder_Sex']][100:2000].loc[nw['Incident_Murder_Sex'] == 1]

#unneeded calculation columns
nw = nw.drop(columns=['Exp', 'sm'])

In [15]:
# Find how many people whose offenses fall into a single incident, where all the offenses are expungeable 
# (Not including attempts). 

x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Inc_Expungeable_Attempts_Not'] == True) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single incident" +
      " in which all the offenses are expungeable" + 
     " if we consider attempts to be unexpungeable.")

date_range(x)

#print(x.sample(n=5)) # grab a random sample to verify
x.Disposition.value_counts(dropna=False)

There are 1247 people with charges eligible for expungement in the NW district of MA, if the person has a single incident in which all the offenses are expungeable if we consider attempts to be unexpungeable.
870 occured more than 3 years before 2020-09-01
27 occured more than 7 years before 2020-09-01


Dismissed at Request of Comm        519
c276s87 finding                     455
Nolle Prosequi                      244
Responsible                         220
Continued w/o Finding               220
Not Responsible                     217
Dismissed Prior to Arraignment      156
Dismissed                           119
NaN                                 111
Dismissed on Payment                 92
Guilty                               36
Delinquent                           14
Dismissed by Court                   14
Responsible Filed                    13
Found Incompetent                     8
Charge Handled as a Civil Charge      5
DYS Committed                         5
CLOSED-NO CHARGES                     3
Delinquent Filed                      3
Dismissed prior to complaint          3
Unagreed Plea                         1
Name: Disposition, dtype: int64

### Question 2

In [16]:
# Find how many people whose offenses fall into a single incident, where all the offenses are expungeable 
# (Including Attempts). 

x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Inc_Expungeable_Attempts_Are'] == True) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()


print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single incident," +
      " in which all the offenses are expungeable and attempts are considered expungeable.")

date_range(x)
#print(x.sample(n=3)) # grab a random sample to verify

x.Disposition.value_counts(dropna=False)

There are 1254 people with charges eligible for expungement in the NW district of MA, if the person has a single incident, in which all the offenses are expungeable and attempts are considered expungeable.
876 occured more than 3 years before 2020-09-01
27 occured more than 7 years before 2020-09-01


Dismissed at Request of Comm        525
c276s87 finding                     459
Nolle Prosequi                      244
Continued w/o Finding               225
Responsible                         221
Not Responsible                     217
Dismissed Prior to Arraignment      156
Dismissed                           119
NaN                                 111
Dismissed on Payment                 93
Guilty                               36
Delinquent                           14
Dismissed by Court                   14
Responsible Filed                    13
Found Incompetent                     8
Charge Handled as a Civil Charge      5
DYS Committed                         5
CLOSED-NO CHARGES                     3
Delinquent Filed                      3
Dismissed prior to complaint          3
Guilty Filed                          1
Unagreed Plea                         1
Name: Disposition, dtype: int64

In [17]:
# Find how many people whose offenses fall into a single incident, where all the offenses are expungeable 
# (Including Attempts). 

x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Inc_Expungeable_Attempts_Not'] == True) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()


print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single incident," +
      " in which all the offenses are expungeable and attempts are not considered expungeable.")

date_range(x)

There are 1247 people with charges eligible for expungement in the NW district of MA, if the person has a single incident, in which all the offenses are expungeable and attempts are not considered expungeable.
870 occured more than 3 years before 2020-09-01
27 occured more than 7 years before 2020-09-01


**Question 2a** -  How many would be eligible if we only limited to only exclude sex-based offenses or murder?

Assuming here we are not limiting based on expungability, but only on if the person has 1 incident, it is related to neither murder nor sex, and the person is under 21. 

In [18]:
x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['murder'] == 0) &
    (nw['sex'] == 0) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has charges unrelated to" +
      " murder or sex.")

date_range(x)

x.Disposition.value_counts(dropna=False)

There are 1796 people with charges eligible for expungement in the NW district of MA, if the person has charges unrelated to murder or sex.
1272 occured more than 3 years before 2020-09-01
42 occured more than 7 years before 2020-09-01


Dismissed at Request of Comm         772
c276s87 finding                      681
Nolle Prosequi                       474
Continued w/o Finding                472
Not Responsible                      369
Responsible                          249
Dismissed Prior to Arraignment       197
Dismissed                            152
NaN                                  151
Dismissed on Payment                  96
Guilty                                91
Delinquent                            37
Responsible Filed                     28
Dismissed by Court                    16
Accord/Satisfaction                   12
Delinquent Filed                      10
Found Incompetent                      8
Not Guilty                             5
Charge Handled as a Civil Charge       5
DYS Committed                          5
Guilty Filed                           4
CLOSED-NO CHARGES                      3
Dismissed prior to complaint           3
DA's Complaint                         2
Guilty on Lesser

In [19]:
x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Incident_Murder_Sex'] == 0) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single incident" +
      " in which all offenses are unrelated to sex or murder")

date_range(x)

There are 1776 people with charges eligible for expungement in the NW district of MA, if the person has a single incident in which all offenses are unrelated to sex or murder
1256 occured more than 3 years before 2020-09-01
37 occured more than 7 years before 2020-09-01


## Question 3: Verdict as Determiner

**Question 3a** -  How many would be eligible based on who has not been found guilty (given current offenses that are eligible for expungement)?

Defining 'guilty' as one of these dispositions: 'Guilty', 'Guilty Filed', 'Guilty on Lesser Included Offense', 'c276s87 finding', 'Agreed Plea', 'Delinquent'

Need more work on this -- there are many dispositions that are null, and many others that indicate a case in process, transferred, etc, that would not indicate "guilty", but also would not indicate "not guilty"

In [20]:
# Guilty -- need more work here on which dispositions to include/excldue
guilty_dispos = ['Guilty', 'Guilty Filed', 'Guilty on Lesser Included Offense', 'c276s87 finding', 
                 'Agreed Plea', 'Delinquent']
nw['guilty'] = nw['Disposition'].isin(guilty_dispos)
nw.loc[nw.Disposition.isnull(), 'guilty'] = None 
nw['Incident_Guilty'] = nw.groupby(['Person ID', 'Offense Date'])['guilty'].transform('max')

In [21]:
# Find how many people are eligble for expungement if they have a verdict other than guilty and their offense is 
# expungeable (Attempts not included)

x = nw.loc[
    (nw['Expungeable'] == 'Yes') &
    (nw['guilty'] == False) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has an expungeable" +
      "offense and has a verdict other than guilty. Attempts counted as not expungeable")

date_range(x)

x.Disposition.value_counts(dropna=False)

There are 2031 people with charges eligible for expungement in the NW district of MA, if the person has an expungeableoffense and has a verdict other than guilty. Attempts counted as not expungeable
1617 occured more than 3 years before 2020-09-01
50 occured more than 7 years before 2020-09-01


Dismissed at Request of Comm        1442
Continued w/o Finding               1237
Nolle Prosequi                       845
Not Responsible                      762
Responsible                          512
Dismissed                            275
Dismissed on Payment                 239
Dismissed Prior to Arraignment       230
Responsible Filed                     84
DYS Committed                         22
Dismissed by Court                    18
Charge Handled as a Civil Charge      17
Accord/Satisfaction                   13
Delinquent Filed                      12
Found Incompetent                      8
Dismissed prior to complaint           5
Case Transferred                       5
CLOSED-NO CHARGES                      4
Valor Act Dispo                        2
Unagreed Plea                          1
Not Guilty                             1
DA's Complaint                         1
Youthful Offender                      1
Name: Disposition, dtype: int64

In [22]:
# Find how many people are eligble for expungement if they have a verdict other than guilty and their offense is 
# expungeable (Attempts included as expungeable)

x = nw.loc[
    (nw['Expungeable'] != 'No') &
    (nw['guilty'] == False) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has an expungeable" +
      "offense and has a verdict other than guilty. Attempts counted as expungeable")

date_range(x)

x.Disposition.value_counts(dropna=False)

There are 2033 people with charges eligible for expungement in the NW district of MA, if the person has an expungeableoffense and has a verdict other than guilty. Attempts counted as expungeable
1619 occured more than 3 years before 2020-09-01
50 occured more than 7 years before 2020-09-01


Dismissed at Request of Comm        1452
Continued w/o Finding               1244
Nolle Prosequi                       849
Not Responsible                      762
Responsible                          512
Dismissed                            275
Dismissed on Payment                 240
Dismissed Prior to Arraignment       230
Responsible Filed                     84
DYS Committed                         22
Dismissed by Court                    18
Charge Handled as a Civil Charge      17
Accord/Satisfaction                   13
Delinquent Filed                      12
Found Incompetent                      8
Dismissed prior to complaint           5
Case Transferred                       5
CLOSED-NO CHARGES                      4
Valor Act Dispo                        2
Unagreed Plea                          1
Not Guilty                             1
DA's Complaint                         1
Youthful Offender                      1
Name: Disposition, dtype: int64

Repeating the above, but using incidents rather than offenses. 

In [23]:
# Find how many people are eligble for expungement if they have a single incident, in which all the offenses are 
# expungeable, and none have a guilty verdict. Attempts are not included. 

x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Inc_Expungeable_Attempts_Not'] == True) &
    (nw['Incident_Guilty'] == False) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21)
]

People_eligible = x['Person ID'].nunique()


print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single" +
      " incident, where all charges are expungeable, and all charge have a disposition other than guilty." +
     " Attempts are considered not expungeable.")

date_range(x)

x.Disposition.value_counts(dropna=False)

There are 848 people with charges eligible for expungement in the NW district of MA, if the person has a single incident, where all charges are expungeable, and all charge have a disposition other than guilty. Attempts are considered not expungeable.
595 occured more than 3 years before 2020-09-01
21 occured more than 7 years before 2020-09-01


Dismissed at Request of Comm        475
Nolle Prosequi                      218
Continued w/o Finding               210
Not Responsible                     199
Responsible                         195
Dismissed Prior to Arraignment      156
Dismissed                           112
Dismissed on Payment                 92
Dismissed by Court                   14
Found Incompetent                     8
Responsible Filed                     8
DYS Committed                         5
Charge Handled as a Civil Charge      4
Dismissed prior to complaint          3
CLOSED-NO CHARGES                     3
Delinquent Filed                      1
Unagreed Plea                         1
Name: Disposition, dtype: int64

In [24]:
# Find how many people are eligble for expungement if they have a single incident, in which all the offenses are 
# expungeable, and none have a guilty verdict. Attempts are not included. 

x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Inc_Expungeable_Attempts_Are'] == True) &
    (nw['Incident_Guilty'] == False) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21)
]

People_eligible = x['Person ID'].nunique()


print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single" +
      " incident, where all charges are expungeable, and all charge have a disposition other than guilty." +
     "Attempts are not expungeable.")

date_range(x)

There are 852 people with charges eligible for expungement in the NW district of MA, if the person has a single incident, where all charges are expungeable, and all charge have a disposition other than guilty.Attempts are not expungeable.
599 occured more than 3 years before 2020-09-01
21 occured more than 7 years before 2020-09-01


**Question 3b** -  How many would be eligible based on who has not been found guilty, except murder or sex offenses?


In [25]:
x = nw.loc[
    (nw['murder'] == 0) &
    (nw['sex'] == 0) &
    (nw['guilty'] == False) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has an offense" +
      " unrelated to sex or murder and has a verdict other than guilty.")

date_range(x)

x.Disposition.value_counts(dropna=False)

There are 2315 people with charges eligible for expungement in the NW district of MA, if the person has an offense unrelated to sex or murder and has a verdict other than guilty.
1868 occured more than 3 years before 2020-09-01
64 occured more than 7 years before 2020-09-01


Dismissed at Request of Comm        1786
Continued w/o Finding               1775
Nolle Prosequi                      1356
Not Responsible                      762
Responsible                          512
Dismissed                            340
Dismissed Prior to Arraignment       281
Dismissed on Payment                 252
Responsible Filed                     84
Accord/Satisfaction                   38
DYS Committed                         25
Delinquent Filed                      22
Dismissed by Court                    21
Charge Handled as a Civil Charge      17
Not Guilty                            17
Case Transferred                       9
Found Incompetent                      8
CLOSED-NO CHARGES                      5
Dismissed prior to complaint           5
Youthful Offender                      4
DA's Complaint                         2
Valor Act Dispo                        2
Unagreed Plea                          1
Name: Disposition, dtype: int64

In [26]:
#Find out how many people would be eligible if they have a single incident, in which all the charges are unrelated to
#sex or murder, and all charges have a disposition other than guilty. Attempts not included. 

x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Incident_Guilty'] == False) &
    (nw['Incident_Murder_Sex'] == False) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single incident" +
      " in which all the charges are unrelated to sex or murder and have a non-guilty verdict.")

date_range(x)

There are 1207 people with charges eligible for expungement in the NW district of MA, if the person has a single incident in which all the charges are unrelated to sex or murder and have a non-guilty verdict.
863 occured more than 3 years before 2020-09-01
25 occured more than 7 years before 2020-09-01
